# Waves And Markers Download Test

In [1]:
import os
import sys
srcpath = os.path.realpath('../../SourceFiles')
sys.path.append(srcpath)
import pyte_visa_utils as pyte

In [2]:
%matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
import matplotlib.pyplot as plt

In [3]:
# Connect to instrument

# Please choose appropriate address:
inst_addr = 'TCPIP::192.168.0.117::5025::SOCKET'

print('Connecting ..')
inst = pyte.open_session(inst_addr)
resp = inst.query("*IDN?")
print('Connected to: ' + resp)


Connecting ..
Connected to: Tabor Electronics,P9484M,000000000000,1.232.0 --slot#: 3, 


In [4]:
# The paranoia-level is either 
# low (0), normal (1) or debug (2)
paranoia_level = 2

print ('Reset instrument ..')

# Reset the instrument
pyte.send_cmd(inst, '*CLS; *RST', paranoia_level)

pyte.send_cmd(inst, ':TRACe:DELete:ALL', paranoia_level)

print('Done')

Reset instrument ..
Done


In [5]:
# Get the model name
resp = inst.query(":SYST:iNF:MODel?")
print("Model: " + resp)

# Infer the natural DAC waveform format
if 'P9082' in resp:
    dac_mode = 8
else:
    dac_mode = 16
print("DAC waveform format: {0} bits-per-point".format(dac_mode))

# Get number of channels
resp = inst.query(":INST:CHAN? MAX")
print("Number of channels: " + resp)
num_channels = int(resp)

# Get the maximal number of segments
resp = inst.query(":TRACe:SELect:SEGMent? MAX")
print("Max segment number: " + resp)
max_seg_number = int(resp)

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.query(":TRACe:FREE?")
arbmem_capacity = int(resp)
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))

max_dac = 2 ** dac_mode - 1
half_dac = max_dac / 2.0

Model: P9484M
DAC waveform format: 16 bits-per-point
Number of channels: 4
Max segment number: 65536
Available memory per DDR: 17,171,480,576 wave-bytes


In [6]:
pyte.send_cmd(inst,":FREQ:RAST 2500MHz")

resp = inst.query(":FREQ:RAST?")
freq = float(resp)
print ("DAC Generate Freq:{0}".format(freq))

DAC Generate Freq:2500000000.0


## Prepare wave-data and markers-data for 12 segments


In [7]:
# Build wave-data and markers-data for 12 segments

print('Build wave-data and markers-data for 12 segments ..')

seglen = 4096 #2 ** 20 # 1MB
num_cycles = [2 ** n for n in range(num_channels)]

waves = [ None for _ in range(num_channels)]
marks = [ None for _ in range(num_channels)]

if dac_mode == 16:
    seg_wave_bytes = seglen * 2
else:
    seg_wave_bytes = seglen
print(seg_wave_bytes)

print (dac_mode)
print (freq)
if (dac_mode == 16) and freq <= 2500000000:
    seg_mark_bytes = seg_wave_bytes // 4
else:
    seg_mark_bytes = seg_wave_bytes // 8
print(seg_mark_bytes)

for ii in range(num_channels):
    ncycles = num_cycles[ii]
    cyclelen = seglen / ncycles
    
    x = np.linspace(start=0, stop=seglen, num=seglen, endpoint=False)
    y = np.fmod(x, cyclelen)
    y = (y <= cyclelen / 2) * max_dac
    y = np.round(y)
    y = np.clip(y, 0, max_dac)
    if dac_mode == 16:
        waves[ii] = y.astype(np.uint16)
    else:
        waves[ii] = y.astype(np.uint8)
    waves[ii].reshape(-1)
    
    if dac_mode == 16:
        cycle_bytes = cyclelen // 4
    else:
        cycle_bytes = cyclelen // 8
        
    x = np.linspace(
        start=0, stop=seg_mark_bytes, num=seg_mark_bytes, endpoint=False)
    y = np.fmod(x, cycle_bytes)
    y = (y <= cycle_bytes / 2) * 255
    y = np.round(y)
    y = np.clip(y, 0, 255)
    marks[ii] = y.astype(np.uint8)
    del x, y
    
print('Done')   
 

Build wave-data and markers-data for 12 segments ..
8192
16
2500000000.0
2048
Done


## Download wave-data and markers-data to segments

In [8]:
# Download segments

# data-type code: 'B' = uint8, 'H' = uint16
wav_dat_type = 'H' if dac_mode == 16 else 'B'
mrk_dat_type = 'B'

for ii in range(num_channels):
    ichan = ii
    channb = ichan + 1
    segnum = ichan % 2 + 1
    wav = waves[ichan]
    mrk = marks[ichan]
    print('Download wave to segment {0} of channel {1}'.format(segnum, channb))
    
    # Select channel
    cmd = ':INST:CHAN {0}'.format(channb)
    pyte.send_cmd(inst, cmd, paranoia_level)
    
    # Define segment
    cmd = ':TRAC:DEF {0}, {1}'.format(segnum, seglen)
    pyte.send_cmd(inst, cmd, paranoia_level)
    
    # Select the segment
    cmd = ':TRAC:SEL {0}'.format(segnum)
    pyte.send_cmd(inst, cmd, paranoia_level)
    
    # Increase the timeout before writing binary-data:
    inst.timeout = 30000
    
    # Send the binary-data with *OPC? added to the beginning of its prefix.
    inst.write_binary_values('*OPC?; :TRAC:DATA', wav, datatype=wav_dat_type)
    
    # Read the response to the *OPC? query that was added to the prefix of the binary data
    resp = inst.read()
    
    # Set normal timeout
    inst.timeout = 10000
    
    resp = inst.query(':SYST:ERR?')
    resp = resp.rstrip()
    if not resp.startswith('0'):
        print('ERROR: "{0}" after writing binary values'.format(resp))
        
    print('Download markers to segment {0} of channel {1}'.format(segnum, channb))
    
    # Increase the timeout before writing binary-data:
    inst.timeout = 30000
    
    # Send the binary-data with *OPC? added to the beginning of its prefix.
    inst.write_binary_values('*OPC?; :MARK:DATA', mrk, datatype=mrk_dat_type)
    
    # Read the response to the *OPC? query that was added to the prefix of the binary data
    resp = inst.read()
    
    # Set normal timeout
    inst.timeout = 10000
    
    resp = inst.query(':SYST:ERR?')
    resp = resp.rstrip()
    if not resp.startswith('0'):
        print('ERROR: "{0}" after writing binary values'.format(resp))

    # Play the specified segment at the selected channel:
    cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(segnum)
    pyte.send_cmd(inst, cmd, paranoia_level)

    # Turn on the output of the selected channel:
    cmd = ':OUTP ON'
    pyte.send_cmd(inst, cmd, paranoia_level)
    
    # Turn on the markers of the selected channel    
    resp = inst.query(':MARK:SEL? MAX')
    resp = resp.rstrip()
    markers_per_chan = int(resp)
    for imarker in range(markers_per_chan):
        cmd = ':MARK:SEL {0}; :MARK:STAT ON'.format(imarker + 1)
        pyte.send_cmd(inst, cmd, paranoia_level)
    
    resp = inst.query(':SYST:ERR?')
    print(resp)

print()

resp = inst.query(':SYST:ERR?')
print(resp)

del waves
del wav

Download wave to segment 1 of channel 1
Download markers to segment 1 of channel 1
ERROR: "219, invalid data size" after writing binary values


C:\Users\Proteus\Desktop\ProteusScripts\Anabelle\Python\Examples\SourceFiles\pyte_visa_utils.py:481: UserWarning: ERR: "219, invalid data size" after CMD: ":SOUR:FUNC:MODE:SEGM 1"
  warnings.warn(wrn_msg)


0, no error
Download wave to segment 2 of channel 2
Download markers to segment 2 of channel 2
ERROR: "219, invalid data size" after writing binary values


C:\Users\Proteus\Desktop\ProteusScripts\Anabelle\Python\Examples\SourceFiles\pyte_visa_utils.py:481: UserWarning: ERR: "219, invalid data size" after CMD: ":SOUR:FUNC:MODE:SEGM 2"
  warnings.warn(wrn_msg)


0, no error
Download wave to segment 1 of channel 3
Download markers to segment 1 of channel 3
ERROR: "219, invalid data size" after writing binary values
0, no error
Download wave to segment 2 of channel 4
Download markers to segment 2 of channel 4
ERROR: "219, invalid data size" after writing binary values
0, no error

0, no error


In [9]:
channb = 1
imarker = 1
cmd = ':INST:CHAN {0}'.format(channb)
pyte.send_cmd(inst, cmd, paranoia_level)
cmd = ':MARK:SEL {0}; :MARK:STAT ON'.format(imarker)
pyte.send_cmd(inst, cmd, paranoia_level)

In [10]:
chanlist = [4,8,12]
imarker = 2
for channb in chanlist:
  if channb <= num_channels:
    cmd = ':INST:CHAN {0}'.format(channb)
    pyte.send_cmd(inst, cmd, paranoia_level)
    cmd = ':MARK:SEL {0}; :MARK:STAT OFF'.format(imarker)
    pyte.send_cmd(inst, cmd, paranoia_level)
    

In [11]:
if 0:
    pyte.send_cmd(inst,":FREQ:RAST 2500MHz")  # 1250

    resp = inst.query(":FREQ:RAST?")
    freq = float(resp)
    print ("DAC Generate Freq:{0}".format(freq))

#### 4. Play the segment 

In [12]:
print('Play the segment ..')

# Play the specified segment at the selected channel:
cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(segnum)
pyte.send_cmd(inst, cmd, paranoia_level)

# Turn on the output of the selected channel:
cmd = ':OUTP ON'
pyte.send_cmd(inst, cmd, paranoia_level)

# Turn on the markers of the selected channel    
resp = inst.query(':MARK:SEL? MAX')
resp = resp.rstrip()
markers_per_chan = int(resp)
for imarker in range(markers_per_chan):
    cmd = ':MARK:SEL {0}; :MARK:STAT ON'.format(imarker + 1)
    pyte.send_cmd(inst, cmd, paranoia_level)

resp = inst.query(':SYST:ERR?')
print(resp)

Play the segment ..
0, no error


In [13]:
if 0:
    cmd = ':INST:CHAN 1'
    pyte.send_cmd(inst, cmd, paranoia_level)
    cmd = ':TRAC:ZERO:ALL'
    pyte.send_cmd(inst, cmd, paranoia_level)

In [14]:
if 0:
    cmd = ':INST:CHAN 2'
    pyte.send_cmd(inst, cmd, paranoia_level)
    cmd = ':TRAC:ZERO:ALL'
    pyte.send_cmd(inst, cmd, paranoia_level)